In [1]:
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url).text
soup = BeautifulSoup(page,'xml')

In [3]:
table = soup.find('table')


In [4]:
#print(table)

In [5]:
postcode = []
borough = []
neighbourhood = []

In [6]:
for tr_cell in table.find_all('tr'):
    counter = 1
    postcode_var = -1
    borough_var = -1
    neighbourhood_var = -1
    for td_cell in tr_cell.find_all('td'):
        if counter == 1:
            postcode_var = td_cell.text
        if counter == 2:
            borough_var = td_cell.text
            tag_a_barough = td_cell.find('a')
        if counter == 3:
            neighbourhood_var = str(td_cell.text).strip()
            tag_a_neighbourhood = td_cell.find('a')
        counter +=1
        if (postcode_var == 'Not assigned' or borough_var == 'Not assigned' or neighbourhood_var == 'Not assigned'):
            continue
    try:
        if ((tag_a_barough is None)) or (tag_a_neighbourhood is None):
            continue
    except:
        pass
    if(postcode_var == -1 or borough_var == -1 or neighbourhood_var == -1):
        continue
    
    postcode.append(postcode_var)
    borough.append(borough_var)
    neighbourhood.append(neighbourhood_var)



In [7]:
unique_code = set(postcode)
postcode_u = []
borough_u = []
neighbourhood_u = []

for pc_u in unique_code:
    p = ''; b = ''; n='';
    for postcode_idx, postcode_element in enumerate(postcode):
        if pc_u == postcode_element:
            p = postcode_element;
            b = borough[postcode_idx]
            if n == '':
                n = neighbourhood[postcode_idx]
            else:
                n = n + ', ' + neighbourhood[postcode_idx]
    postcode_u.append(p)
    borough_u.append(b)
    neighbourhood_u.append(n)

In [8]:
toronto = {'postcode':postcode_u,'borough':borough_u,'neighbourhood':neighbourhood_u}
df_canada = pd.DataFrame.from_dict(toronto)


In [9]:
df_canada.head(10)

,postcode,borough,neighbourhood
0,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Sou..."
1,M4E,East Toronto,The Beaches
2,M1K,Scarborough,"Ionview, Kennedy Park"
3,M8Y,Etobicoke,"Humber Bay, Mimico NE, Old Mill South, The Que..."
4,M9M,North York,"Emery, Humberlea"
5,M6H,West Toronto,Dovercourt Village
6,M3H,North York,"Bathurst Manor, Wilson Heights"
7,M1B,Scarborough,"Rouge, Malvern"
8,M1M,Scarborough,"Cliffcrest, Cliffside"
9,M5A,Downtown Toronto,"Harbourfront, Regent Park"


In [12]:
df_coord = pd.read_csv('./Geospatial_Coordinates.csv')


In [13]:
df_coord.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
df_loc = df_coord.rename(columns={"Postal Code": "postcode"})

In [15]:
df_loc.head(5)

,postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
df_canada.head(5)

,postcode,borough,neighbourhood
0,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Sou..."
1,M4E,East Toronto,The Beaches
2,M1K,Scarborough,"Ionview, Kennedy Park"
3,M8Y,Etobicoke,"Humber Bay, Mimico NE, Old Mill South, The Que..."
4,M9M,North York,"Emery, Humberlea"


In [23]:
result = pd.merge(df_canada, df_loc, on='postcode')

In [24]:
result.head(5)

,postcode,borough,neighbourhood,Latitude,Longitude
0,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Sou...",43.628947,-79.394420
1,M4E,East Toronto,The Beaches,43.676357,-79.293031
2,M1K,Scarborough,"Ionview, Kennedy Park",43.727929,-79.262029
3,M8Y,Etobicoke,"Humber Bay, Mimico NE, Old Mill South, The Que...",43.636258,-79.498509
4,M9M,North York,"Emery, Humberlea",43.724766,-79.532242
